In [1]:
# !pip install tk_html_widgets

In [2]:
import question_constructor as qc
import SRQuestion_generator as sg
import ANSQuestion_generator as ag
import MathQuestion_generator as mag
import MemoryQuestion_generator as meg
import data_interaction as di
import tkinter as tk
from tkinter import ttk
from PIL import Image, ImageTk
import os
import threading
from tk_html_widgets import HTMLLabel
import numpy as np
import time

In [3]:
def test_instruction(frame, instruction):
    instruction_label = HTMLLabel(frame, html=instruction, width=75, height=20)
    instruction_label.configure(bg="white")
    instruction_label.place(relx=0.5, rely=0.5, anchor=tk.CENTER)
    
    timer_label = HTMLLabel(frame, html="<h3 style='background-color:white;'>Questions not loaded yet...</h3>", width=75)
    timer_label.configure(bg="white")
    timer_label.place(relx=0.5, rely=1, anchor=tk.CENTER)
    
    return instruction_label, timer_label

In [4]:
def test_set_up():
    ANST_instruction = f"""
    <div style="display:grid; place-items:center; background-color:white;">
    <h2>ANS Test</h2>
    <div style='font-size:12px;'>
    <p>The test challenges participants to select the image with a higher count of randomly placed dots by pressing the according arrow key<strong>LEFT / RIGHT</strong>.</p>
    <p>These images flash on the screen for a duration of 0.75 seconds, with a 3-second time limit given for each response.</p>
    <p>Between each question, there is a 1.5-second interval to prepare for the next one.</p>
    <p>Participant's accuracy and response are recorded.</p>
    </div>
    </div>
    """
    
    MathT_instruction = f"""
    <div style="display:grid; place-items:center; background-color:white;">
    <h2>Math Test</h2>
    <div style='font-size:12px;'>
    <p>The test comprises 15 mathematical questions, each involving a sequence of calculation steps displayed for 2 seconds per step.</p>
    <p>Participants are required to memorize these steps and input the final calculation result after the steps are no longer visible.</p>
    <p>Between each question, there is a 1.5-second interval to prepare for the next one.</p>
    p>Participant's accuracy and response are recorded.</p>
    </div>
    </div>
    """
    
    MemoryT_instruction = f"""
    <div style="display:grid; place-items:center; background-color:white;">
    <h2>Memory Test</h2>
    <div style='font-size:12px;'>
    <p>The Memory Test is structured around four main questions, each comprising a matrix of images.</p>
    <p>Participants are tasked with memorizing as many details as possible about the colors and relative positions of the images within a 20-second timeframe.</p>
    <p>Following this memorization phase, participants answer five related questions each with a 5-second time limit to test their recall of the images.</p>
    p>Participant's accuracy and response are recorded.</p>
    </div>
    </div>
    """
    
    SRT_instruction = f"""
    <div style="display:grid; place-items:center; background-color:white;">
    <h2>Spatial Reasoning Test</h2>
    <div style='font-size:12px;'>
    <p>Participants are presented with a series of 9 spatial reasoning questions, each involving a randomly generated three-dimensional cybe arrangement.</p>
    <p>The task is to identify, from four options (A, B, C, D), which two-dimensional image cannot be obtained by rotating the given three-dimensional figure.</p>
    <p>A time limit of 25 seconds is allocated for answering each question, aiming to assess not only accuracy but also the speed of spatial reasoning.</p>
    p>Participant's accuracy and response are recorded.</p>
    </div>
    </div>
    """
    
    ANST_labels = test_instruction(ANSTest_frame, ANST_instruction)
    MathT_labels = test_instruction(MathTest_frame, MathT_instruction)
    MemoryT_labels = test_instruction(MemoryTest_frame, MemoryT_instruction)
    SRT_labels = test_instruction(SRTest_frame, SRT_instruction)
    
    image_list, num_points_list, answer_list= ag.ANSQuestion_bank(60)
    ANST_dict["question_image_list"]=image_list
    ANST_dict["num_left_list"]=[nums[0] for nums in num_points_list]
    ANST_dict["num_right_list"]=[nums[1] for nums in num_points_list]
    ANST_dict["ratio_list"] = [nums[0] / nums[1] for nums in num_points_list]
    ANST_dict["question_answer_list"]=answer_list

    ANSTest(ANSTest_frame, ANST_labels)
    
    print("ANSTest loaded")
    
    equation_list, answer_list = mag.MathQuestion_bank(60)
    MathT_dict["question_equation_list"]=equation_list
    MathT_dict["question_answer_list"]=answer_list
    
    print(MathT_dict["question_equation_list"])
    MathTest(MathTest_frame, MathT_labels)
    
    print("MathTest loaded")
    
    question_list = meg.MemoryQuestion_bank()
    MemoryT_dict["description_image_list"]= [question[0] for question in question_list]
    MemoryT_dict["question_description_list"]= [subquestion[0] for question in question_list for subquestion in question[1]]
    MemoryT_dict["question_option_list"]= [subquestion[1] for question in question_list for subquestion in question[1]]
    MemoryT_dict["question_answer_list"]= [subquestion[2] for question in question_list for subquestion in question[1]]
    MemoryT_dict["question_image_list"]= [subquestion[3] for question in question_list for subquestion in question[1]]
    
    print(MemoryT_dict["description_image_list"])
    MemoryTest(MemoryTest_frame, MemoryT_labels)
    
    print("MemoryTest loaded")
    
    image_list, options_list, answer_list, grid_size_list = sg.SRQuestion_bank(60)
    SRT_dict["question_3d_image_list"]=image_list
    SRT_dict["question_options_list"]=options_list
    SRT_dict["question_answer_list"]=answer_list
    SRT_dict["grid_size_list"]=grid_size_list
    
    SRTest(SRTest_frame, SRT_labels)
    
    print("SRTest loaded")

In [5]:
def SRTest(SRTest_frame, SRT_labels):
    form_id = "1FAIpQLScVhZDFl3MBSRnaANngjmYk_5Ej0icxzYqc2ZlyPkcw2MrcFw"
    sheet_id = "1DBMUZZLjtrP1ZDCP7AH0vBK-nyn_AuGgGjFKrsik8M4"

    question_list = []
    
    def background():
        while SRTest_frame.winfo_viewable() == 0:
            time.sleep(0.1)
        for t in range(5):
            SRT_labels[1].set_html(f"<h3>Test will start in {5-t} seconds.</h3>")
            time.sleep(1)
        SRT_labels[0].place_forget()
        SRT_labels[1].place_forget()
        
        idx = 1
        question_num = len(question_list)
        bar_description = tk.Label(progress_indicator, text=f"Q {idx}/{question_num} :", bg="white")
        progress_bar = ttk.Progressbar(progress_indicator, orient="horizontal", length=100, mode="determinate")
        timer = tk.Label(progress_indicator, text="Question not fully displayed", font=("Helvetica", 12), bg="white")
        bar_description.grid(row=0, column=0)
        progress_bar.grid(row=0, column=1)
        timer.grid(row=1, column=0, columnspan=2)
        while idx <= question_num:
            question = question_list[idx-1]
            if question.shown == False:
                question.display_question()
            elif question.correctness != None or question.time_up==True:
                idx += 1
                bar_description["text"] = text=f"Q {idx}/{question_num} :"
                progress_bar["value"] = idx*100/question_num
                timer["text"] = "Question not fully displayed"
            else:
                timer["text"] = f"You have {question.timeout-question.get_time():.1f} seconds left"
                question.check_timeout()
        progress_indicator.destroy()
        
        get_result(SRTest_frame, question_list, SRT_dict, form_id, sheet_id)
        
        time.sleep(3)
        
        return root.after(0, SRTest_frame.destroy)

    for i in range(len(SRT_dict["question_3d_image_list"])):
        question = qc.SpatialReasoningQuestion(SRTest_frame,
                                               "Which of the views (a-d) can not be made by rotating the cube arrangement shown?",
                                               SRT_dict["question_options_list"][i],
                                               SRT_dict["question_answer_list"][i],
                                               SRT_dict["question_3d_image_list"][i],
                                               timeout=25)
        question_list.append(question)

    progress_indicator = tk.Frame(SRTest_frame, bg="white")
    progress_indicator.place(relwidth=0.5)
    threading.Thread(target=background, daemon=True).start()
    

In [6]:
def MemoryTest(MemoryTest_frame, MemoryT_labels):
    
    form_id = "1FAIpQLSe84e3y21rKnMHL0_j1qVyVIPc8Z4vsnmTLkQrDK77vXx4xRg"
    sheet_id = "1Gb0oRYE3iq47QvIrLnrEC7vypCDfn5QccEqoWORhNWs"

    question_list = []
    subquestion_list = []
    
    def background():
        while MemoryTest_frame.winfo_viewable() == 0:
            time.sleep(0.1)
            
        for t in range(5):
            MemoryT_labels[1].set_html(f"<h3>Test will start in {5-t} seconds.</h3>")
            time.sleep(1)
        MemoryT_labels[0].place_forget()
        MemoryT_labels[1].place_forget()
        
        i = 1
        idx = 1
        
        question_num = len(subquestion_list)
        bar_description = tk.Label(progress_indicator, text=f"Q {idx}/{question_num} :", bg="white")
        progress_bar = ttk.Progressbar(progress_indicator, orient="horizontal", length=100, mode="determinate")
        timer = tk.Label(progress_indicator, text="Question not fully displayed", font=("Helvetica", 12), bg="white")
        bar_description.grid(row=0, column=0)
        progress_bar.grid(row=0, column=1)
        timer.grid(row=1, column=0, columnspan=2)
        while idx <= question_num:
            question = question_list[int(idx/5)]
            if question.shown == False:
                question.display_question()
            else:
                timer["text"] = f"You have {question.timeout-question.get_time():.1f} seconds left"
                question.check_timeout()
            if question.time_up == True:
                while i <= 5:
                    subquestion = subquestion_list[idx-1]
                    if subquestion.shown == False:
                        subquestion.display_question()
                    elif subquestion.correctness != None or subquestion.time_up==True:
                        i += 1
                        idx += 1
                        bar_description["text"] = text=f"Q {idx}/{question_num} :"
                        progress_bar["value"] = idx*100/question_num
                        timer["text"] = "Question not fully displayed"
                    else:
                        timer["text"] = f"You have {subquestion.timeout-subquestion.get_time():.1f} seconds left"
                        subquestion.check_timeout()
                i = 1
        progress_indicator.destroy()
        
        get_result(MemoryTest_frame, subquestion_list, MemoryT_dict, form_id, sheet_id)
        
        time.sleep(3)
        
        return root.after(0, MemoryTest_frame.destroy)

    for i in range(len(MemoryT_dict["description_image_list"])):
        question =qc.Question(MemoryTest_frame,
                             "You will have 20s to remember this picture.",
                             MemoryT_dict["description_image_list"][i],
                             timeout=20)
        question_list.append(question)
    for i in range(len(MemoryT_dict["question_image_list"])):
        subquestion = qc.MemoryQuestion(MemoryTest_frame,
                                        MemoryT_dict["question_description_list"][i],
                                        MemoryT_dict["question_option_list"][i],
                                        MemoryT_dict["question_answer_list"][i],
                                        MemoryT_dict["question_image_list"][i],
                                        timeout=5)
        subquestion_list.append(subquestion)

    progress_indicator = tk.Frame(MemoryTest_frame, bg="white")
    progress_indicator.place(relwidth=0.5)
    threading.Thread(target=background, daemon=True).start()
    

In [7]:
def MathTest(MathTest_frame, MathT_labels):

    form_id = "1FAIpQLSeJ4SykSiZi5Y4_NnOtj2hLKR9jYYWd-mZSDHQWoIr39uH_1A"
    sheet_id = "1QJy2MaWVaj95mrJ-kCtuIzV-2a1hcpL2N0a2yeb33-8"

    question_list = []
    
    def background():
        while MathTest_frame.winfo_viewable() == 0:
            time.sleep(0.1)
            
        for t in range(5):
            MathT_labels[1].set_html(f"<h3>Test will start in {5-t} seconds.</h3>")
            time.sleep(1)
        MathT_labels[0].place_forget()
        MathT_labels[1].place_forget()
        
        idx = 1
        question_num = len(question_list)
        bar_description = tk.Label(progress_indicator, text=f"Q {idx}/{question_num} :", bg="white")
        progress_bar = ttk.Progressbar(progress_indicator, orient="horizontal", length=100, mode="determinate")
        timer = tk.Label(progress_indicator, text="Question not fully displayed", font=("Helvetica", 12), bg="white")
        bar_description.grid(row=0, column=0)
        progress_bar.grid(row=0, column=1)
        timer.grid(row=1, column=0, columnspan=2)
        while idx <= question_num:
            question = question_list[idx-1]
            if question.shown == False:
                question.display_question()
            elif question.correctness != None or question.time_up==True:
                idx += 1
                bar_description["text"] = text=f"Q {idx}/{question_num} :"
                progress_bar["value"] = idx*100/question_num
                timer["text"] = "Question not fully displayed"
            else:
                timer["text"] = f"You have {question.timeout-question.get_time():.1f} seconds left"
                question.check_timeout()
        progress_indicator.destroy()
        
        get_result(MathTest_frame, question_list, MathT_dict, form_id, sheet_id)
        
        time.sleep(3)
        
        return root.after(0, MathTest_frame.destroy)

    for i in range(len(MathT_dict["question_equation_list"])):
        question = qc.MathQuestion(MathTest_frame,
                                "Remember the calculation steps shown in sequence, write the final result after they vanish.",
                                MathT_dict["question_equation_list"][i],
                                MathT_dict["question_answer_list"][i],
                                timeout=15)
        question_list.append(question)

    progress_indicator = tk.Frame(MathTest_frame, bg="white")
    progress_indicator.place(relwidth=0.5)
    threading.Thread(target=background, daemon=True).start()
    

In [8]:
def ANSTest(ANSTest_frame, ANST_labels):
    
    form_id = "1fzRQj3POD2T0OVnbIW-XXhdEqYp9LxvA0_1XhT51AE0"
    sheet_id = "19uvDxv2Vhb7-bVsUtOyiaTfbkiMAfMGp1R7FqhDixho"
    
    question_list = []
    
    def background():
        while ANSTest_frame.winfo_viewable() == 0:
            time.sleep(0.1)
            
        for t in range(5):
            ANST_labels[1].set_html(f"<h3>Test will start in {5-t} seconds.</h3>")
            time.sleep(1)
        ANST_labels[0].place_forget()
        ANST_labels[1].place_forget()
        
        idx = 1
        question_num = 64
        bar_description = tk.Label(progress_indicator, text=f"Q {idx}/{question_num} :", bg="white")
        progress_bar = ttk.Progressbar(progress_indicator, orient="horizontal", length=100, mode="determinate")
        timer = tk.Label(progress_indicator, text="Question not fully displayed", font=("Helvetica", 12), bg="white")
        bar_description.grid(row=0, column=0)
        progress_bar.grid(row=0, column=1)
        timer.grid(row=1, column=0, columnspan=2)
        while idx <= question_num:
            question = question_list[idx-1]
            if question.shown == False:
                question.display_question()
            elif question.correctness != None or question.time_up==True:
                time.sleep(1.5)
                idx += 1
                bar_description["text"] = text=f"Q {idx}/{question_num} :"
                progress_bar["value"] = idx*100/question_num
                timer["text"] = "Question not fully displayed"
            else:
                timer["text"] = f"You have {question.timeout-question.get_time():.1f} seconds left"
                question.check_timeout()
        progress_indicator.destroy()
        
        get_result(ANSTest_frame, question_list, ANST_dict, form_id, sheet_id)
        
        time.sleep(3)
        
        return root.after(0, ANSTest_frame.destroy)
    
    for i in range(len(ANST_dict["question_image_list"])):
        question = qc.ANSQuestion(ANSTest_frame,
                                  "Press the left or right arrow key based on which image has more dots after dots disappear.",
                                  ANST_dict["question_answer_list"][i],
                                  ANST_dict["question_image_list"][i],
                                  timeout=3)
        question_list.append(question)
    
    progress_indicator = tk.Frame(ANSTest_frame, bg="white")
    progress_indicator.place(relwidth=0.5)
    threading.Thread(target=background, daemon=True).start()

In [9]:
def get_consent(consent_frame):                
    consent_text = """
    <div style="display:grid; place-items:center; background-color:white;">
    <h2>Please read:</h2>
    <div style='font-size:12px;'>
    <p>We wish to record your response data to an anonymised public data repository.</p>
    <p>Your data will be used for educational teaching purposes practising data analysis and visualisation.</p>
    <p>Please click <b>Yes</b> below if you consent to the upload.</p>
    </div>
    </div>
    """
    
    consent_label = HTMLLabel(consent_frame, html=consent_text, height=12)
    consent_label.configure(bg="white")
    
    def consent_submission(consent):
        consent_frame.destroy()
        main_dict["consent"]=consent
    
    yes_btn = tk.Button(consent_frame, text="Yes", width=15, command=lambda: consent_submission(True), font=("Helvetica", 15), bg="#28a745", fg="white")
    no_btn = tk.Button(consent_frame, text="No", width=15, command=lambda: consent_submission(False), font=("Helvetica", 15), bg="#dc3545", fg="white")
    
    consent_label.grid(row=0, column=0, columnspan=2)
    yes_btn.grid(row=1, column=0, sticky="E")
    no_btn.grid(row=1, column=1, sticky="W")

In [10]:
def get_info(info_frame):
    instruction = """
        <div style="display: grid; place-items: center; background-color:white;">
        <h2>Enter your anonymised ID & personal information</h2>
        <div style='font-size:12px;'>
        <h4>Please read the following instructions</h4>
        <p>To generate an anonymous 4-letter unique user identifier please enter:</p>
        <ul>
            <li>Two letters based on the initials (first and last name) of a childhood friend</li>
            <li>Two letters based on the initials (first and last name) of a favourite actor / actress</li>
        </ul>
        <p>e.g., if your friend was Charlie Brown and film star was Tom Cruise, then your ID would be CBTC</p>
        </div>
        </div>
        """
    
    def info_submission():
        main_dict["user_id"]=id_input.get()
        main_dict["gender"]=gender_input.get()
        main_dict["age"]=age_input.get()
        main_dict["sports"]=sports_input.get()
        main_dict["tiredness"]=tiredness_input.get()
        info_frame.destroy()
    
    instruction_label = HTMLLabel(info_frame, html=instruction, height=12)
    instruction_label.configure(bg="white")
    instruction_label.grid(row=0, column=0, columnspan=2)
    
    def enforce_policy(*args):
        value = user_id.get()
        if len(value) > 4: 
            user_id.set(value[:4])
        else:
            user_id.set(value.upper())
    
    user_id = tk.StringVar()
    user_id.trace_add("write", enforce_policy)
    
    id_label = tk.Label(info_frame, text="User ID:", bg="white", font=("Helvetica", 12))
    id_input = tk.Entry(info_frame, textvariable=user_id, width=22, bg="white", font=("Helvetica", 12))
    
    gender_label = tk.Label(info_frame, text="Gender:", bg="white", font=("Helvetica", 12))
    gender_input = ttk.Combobox(info_frame, values=["Male", "Female", "Other"], width=20, font=("Helvetica", 12))
    
    age_label = tk.Label(info_frame, text="Age:", bg="white", font=("Helvetica", 12))
    age_input = tk.Spinbox(info_frame, from_=0, to=120, width=21, bg="white", font=("Helvetica", 12))
    
    sports_label = tk.Label(info_frame, text="How often you do sports:", bg="white", font=("Helvetica", 12))
    sports_input = ttk.Combobox(info_frame, values=["Frequently(>5 days per week)", "Often(3-4 days per week)", "Sometime(1-2 days per week)", "Never(0 days per week)"], width=20, font=("Helvetica", 12))
    
    tiredness_label = tk.Label(info_frame, text="How tired you are:", bg="white", font=("Helvetica", 12))
    tiredness_input = tk.Scale(info_frame, from_=0, to=10, orient="horizontal", resolution=1, length=200, bg="white", font=("Helvetica", 12))
    
    submission_btn = tk.Button(info_frame, text="Submit", width=15, command=info_submission, font=("Helvetica", 15), bg="#28a745", fg="white")
    
    id_label.grid(row=1, column=0, sticky="E")
    id_input.grid(row=1, column=1, sticky="W", pady=5)
    gender_label.grid(row=2, column=0, sticky="E")
    gender_input.grid(row=2, column=1, sticky="W", pady=5)
    age_label.grid(row=3, column=0, sticky="E")
    age_input.grid(row=3, column=1, sticky="W", pady=5)
    sports_label.grid(row=4, column=0, sticky="E")
    sports_input.grid(row=4, column=1, sticky="W", pady=5)
    tiredness_label.grid(row=5, column=0, sticky="E")
    tiredness_input.grid(row=5, column=1, sticky="W", pady=5)
    submission_btn.grid(row=6, column=0, columnspan=2, pady=5)    

In [11]:
def percentile_rank_calculator(total_score, sheet_id):
    score_list = np.array(di.get_data(["total_score"], sheet_id)["total_score"]).astype(int)
    count = sum(1 for score in score_list if score < total_score)
    index = count
    percentage = (index / len(score_list)) * 100
    return percentage

In [12]:
def get_result(frame, question_list, Test_dict, form_id, sheet_id):
    for question in question_list:
        if question.correctness:
            Test_dict["total_score"] += 1
            Test_dict["score_list"].append(1)
        else:
            Test_dict["score_list"].append(0)
        Test_dict["total_time"] += question.get_time()
        Test_dict["time_list"].append(question.get_time())
        
    di.send_data(Main_dict|Test_dict, form_id)
    
    total_score = Test_dict["total_score"]
    total_questions = len(Test_dict["question_answer_list"])
    percentile_rank = percentile_rank_calculator(total_score, sheet_id)
    result = f"""
    <div style="text-align: center; background-color: white; font-size: 12px;">
    <p>You have got <strong>{total_score}/{total_questions}</strong>.</p>
    <p>You have beaten <strong>{percentile_rank}%</strong> of people in dataset.</p>
    </div>
    """

    result_label = HTMLLabel(frame, html=result, height=12)
    result_label.pack()
    result_label.configure(bg="white")
    

In [13]:
def opening(root):
    consent_frame = tk.Frame(root, bg="white")
    info_frame = tk.Frame(root, bg="white")
    frames.append(consent_frame)
    frames.append(info_frame)
    get_consent(consent_frame)
    get_info(info_frame)
    

In [14]:
def ending(root):
    ending_frame = tk.Frame(root, bg="white", width=600, height=800)
    frames.append(ending_frame)
    
    thank_text = f"""
    <div style="text-align: center; background-color:white;"">
    <h2>Thank you for taking the test!</h2>
    <h2>Wish you a nice day!</h2>
    <p>Please contact <strong>a.fedorec@ucl.ac.uk</strong> if you have any questions or concerns.</p>
    </div>
    """

    thank_label = HTMLLabel(ending_frame, html=thank_text, height=12)
    thank_label.pack()
    thank_label.configure(bg="white")

In [15]:
def background():
    for frame in frames:
        frame.place(relx=0.5, rely=0.5, anchor=tk.CENTER)
        while frame.winfo_exists() == 1:
            if main_dict["consent"] == False:
                root.after(0, root.destroy)

frames=[]

root = tk.Tk()
root.configure(bg='white')
root.attributes('-topmost', True)
root.title("Cognitive Test")

screen_width = root.winfo_screenwidth()
screen_height = root.winfo_screenheight()
x = (screen_width / 2) - (800 / 2)
y = (screen_height / 2) - (900 / 2)
root.geometry('%dx%d+%d+%d' % (800, 900, x, y))

main_dict = {
    "consent":None,
    "user_id":None,
    "age":None,
    "gender":None,
    "sports":None,
    "tiredness":None,
}

ANST_dict = {
    "total_score":0,
    "total_time":0,
    "question_image_list":[],
    "num_left_list":[],
    "num_right_list":[],
    "ratio_list":[],
    "question_answer_list":[],
    "score_list":[],
    "time_list":[]
}

MathT_dict = {
    "total_score":0,
    "total_time":0,
    "question_equation_list":[],
    "question_answer_list":[],
    "score_list":[],
    "time_list":[]
}

MemoryT_dict = {
    "total_score":0,
    "total_time":0,
    "description_image_list":[],
    "question_image_list":[],
    "question_answer_list":[],
    "score_list":[],
    "time_list":[]
}

SRT_dict = {
    "total_score":0,
    "total_time":0,
    "question_3d_image_list":[],
    "question_options_list":[],
    "question_answer_list":[],
    "grid_size_list":[],
    "score_list":[],
    "time_list":[]
}

opening(root)
ANSTest_frame = tk.Frame(root, bg="white", width=600, height=900)
frames.append(ANSTest_frame)
MathTest_frame = tk.Frame(root, bg="white", width=600, height=900)
frames.append(MathTest_frame)
MemoryTest_frame = tk.Frame(root, bg="white", width=600, height=900)
frames.append(MemoryTest_frame)
SRTest_frame = tk.Frame(root, bg="white", width=600, height=900)
frames.append(SRTest_frame)

ending(root)

threading.Thread(target=test_set_up, daemon=True).start()

threading.Thread(target=background, daemon=True).start()

root.mainloop()


ANSTest loaded
[['2', '×5'], ['9', '-18'], ['3', '-24'], ['35', '+4'], ['45', '×7'], ['1', '×5', '÷1'], ['12', '+14', '×11'], ['11', '-30', '÷1'], ['40', '+19', '×13'], ['33', '+18', '×10'], ['3', '-3', '÷8', '×9'], ['6', '÷3', '÷1', '÷2'], ['3', '+11', '-27', '+17'], ['40', '×3', '+3', '+26'], ['39', '÷13', '+35', '-50']]
MathTest loaded
['./Memory_Test/Figures/Description_img/MemoryQ_1.png', './Memory_Test/Figures/Description_img/MemoryQ_2.png', './Memory_Test/Figures/Description_img/MemoryQ_3.png', './Memory_Test/Figures/Description_img/MemoryQ_4.png']
MemoryTest loaded
SRTest loaded


Exception in thread Thread-8 (background):
Traceback (most recent call last):
  File "C:\Program Files\Anaconda3\lib\threading.py", line 1016, in _bootstrap_inner
Exception in thread Thread-7 (background):
Traceback (most recent call last):
  File "C:\Program Files\Anaconda3\lib\threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "C:\Program Files\Anaconda3\lib\threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\zcbtbao\AppData\Local\Temp\ipykernel_15256\2410616409.py", line 9, in background
    self.run()
  File "C:\Program Files\Anaconda3\lib\threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\zcbtbao\AppData\Local\Temp\ipykernel_15256\3081442157.py", line 9, in background
  File "C:\Program Files\Anaconda3\lib\tkinter\__init__.py", line 1267, in winfo_viewable
  File "C:\Program Files\Anaconda3\lib\tkinter\__init__.py", line 1267, in winfo_viewable
    self.tk.call('winfo', 'viewabl